## Simple  Linear Regression

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import time

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/willn9/price_is_right/main/02_area_price_000.csv") # github
# df = pd.read_csv("02_area_price_000.csv") # local

In [ ]:
df # Note that the price is in $000, i.e., a value of 350 means that the house price is $350,000

In [ ]:
# Extract x (area) and y (price)
x = df["area"].values
y = df["price"].values

In [ ]:
type(x[0]), type(y[0])

In [ ]:
# Construct X matrix with a bias term (column of ones)
X = np.column_stack((np.ones(len(x)), x))
X

In [ ]:
# Apply normal equation: β = (Xᵀ·X)⁻¹·Xᵀ·y
XT_X = X.T.dot(X)
XT_y = X.T.dot(y)
beta = np.linalg.inv(XT_X).dot(XT_y)

In [ ]:
# Results
B0, B1 = beta
print(f"B0 (Intercept): {B0:.2f}")
print(f"B1 (Slope): {B1:.2f}")

In [ ]:
# Predict prices using the linear model
y_pred = B0 + B1 * x

# Plot actual vs. predicted
plt.scatter(x, y, color='blue', label='Actual Data')
plt.plot(x, y_pred, color='red', label='Linear Fit')
plt.xlabel("Area (sq ft)")
plt.ylabel("Price")
plt.title("Linear Regression: House Area vs. Price")
plt.legend()
plt.grid(True)
plt.show()

## Price is Right Game!

In [ ]:
# Tracking data
areas = []
predicted_prices = []
guesses = []
errors = []
squared_errors = []

# Game duration
t_minutes = 3 # set minutes
duration = 60  * t_minutes
start_time = time.time()

# Valid exit phrases
stop_commands = ["exit", "stop", "quit", "^c"]

print(f"🎮 Price is Right Regression Challenge\nYou have {t_minutes} minutes to guess house prices based on area. Let's go!")

while time.time() - start_time < duration:
    # Generate house area in range [400, 2500]
    area = np.random.randint(400, 2500)
    predicted_price = B0 + B1 * area
    print(f"\n🏠 House Area: {area} sq ft")

    guess_input = input("💰 Your guess for the house price in $1000, i.e, if your guess $350,000 - you should enter 350 (or type 'stop' to quit): ")

    if guess_input.lower() in stop_commands:
        print("🛑 Game stopped by user.")
        break

    try:
        guess = float(guess_input)
        error = guess - predicted_price
        se = error**2

        areas.append(area)
        predicted_prices.append(predicted_price)
        guesses.append(guess)
        errors.append(error)
        squared_errors.append(se)

        remaining_time = int(duration - (time.time() - start_time))
        print(f"📊 Predicted: ${1000*predicted_price:.2f} | Your Guess: ${1000*guess:.2f} | Error: ${1000*error:.2f} | ⏱️ Remaining Time: {remaining_time} seconds")

    except ValueError:
        print("❌ Invalid input. Please enter a number or a valid stop command.")

print("\n⏰ Time's up! Execute the Next Cells to Know your Score \n and See how you did:")


In [ ]:
# Summary table
results = pd.DataFrame({
    "Round": range(1, len(guesses) + 1),
    "Area (sq ft)": areas,
    "Your Guess ($)": guesses,
    "Model Prediction ($)": predicted_prices,
    "Error ($)": errors,
    "Squared Error": squared_errors
})

# Metrics
SSR = sum(squared_errors)
MSE = SSR / len(squared_errors) if squared_errors else 0
mean_prediction = np.mean(predicted_prices)
SST = sum((pred - mean_prediction)**2 for pred in predicted_prices)
R2 = 1 - SSR / SST if SST != 0 else 0

score = round(1000 * R2 * np.log1p(len(guesses)), 2)


# Summary row
summary = pd.DataFrame({
    "🧾 Summary of": [f"{len(areas)} rounds"],
    "Area (sq ft)": [f"Avg: {np.mean(areas):.2f}"],
    "Your Guess ($)": [f"Avg: {np.mean(guesses):.2f}"],
    "Model Prediction ($)": [f"Avg: {mean_prediction:.2f}"],
    "Error ($)": [f"Total SSR: {SSR:.2f}"],
    "Squared Error | Accuracy": [f"MSE: {MSE:.2f} | R²: {R2:.4f}"],
    "Score": [f"🏆 Score: {score}"]
})

display(results)
summary

In [ ]:
# Plot guesses vs. prediction
plt.figure(figsize=(10,6))
plt.plot(results["Round"], results["Your Guess ($)"], marker='o', label='Your Guesses', color='blue')
plt.plot(results["Round"], results["Model Prediction ($)"], marker='x', linestyle='--', label='Model Prediction', color='red')
plt.xlabel("Round")
plt.ylabel("Price in 1000($)")
plt.title("Price Guess vs. Predicted Price")
plt.legend()
plt.grid(True)
plt.show()